<a href="https://colab.research.google.com/github/plthiyagu/AI-Engineering/blob/master/14-Pandas/Feature_Engineering_Using_Pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.analyticsvidhya.com/blog/2020/12/feature-engineering-using-pandas-for-beginners/?utm_source=feedburner&utm_medium=email&utm_campaign=Feed%3A+AnalyticsVidhya+%28Analytics+Vidhya%29

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [6]:
data=pd.read_csv('https://raw.githubusercontent.com/plthiyagu/Personnel/master/Dataset/Big%20Mart%20Sales%20Prediction.csv')

Getting to know the data:


In [7]:
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


replace() for Label Encoding:


In [ ]:
data['Outlet_Location_Type_Encoded']  = data['Outlet_Location_Type'] \
                                            .replace({'Tier 1': 1, 'Tier 2': 2, 'Tier 3': 3})

data[['Outlet_Location_Type', 'Outlet_Location_Type_Encoded']].head()


get_dummies() for One Hot Encoding


In [8]:
Outlet_Type_Dumm = pd.get_dummies(data=data['Outlet_Type'], columns=['Outlet_Type'], drop_first=True)

pd.concat([data['Outlet_Type'], Outlet_Type_Dumm], axis=1).head()

,Outlet_Type,Supermarket Type1,Supermarket Type2,Supermarket Type3
0,Supermarket Type1,1,0,0
1,Supermarket Type2,0,1,0
2,Supermarket Type1,1,0,0
3,Grocery Store,0,0,0
4,Supermarket Type1,1,0,0


cut() and qcut() for Binning

Binning is a technique of grouping together values of continuous variables into n number of bins. Binning can also be called a discretization technique as we are dividing or partitioning a continuous variable to a discrete variable.

Sometimes using a discrete variable than a continuous variable is better for some machine learning algorithms. For example: having a continuous variable like age can be used better and more interpretable if we convert it into age brackets. Binning continuous variables also help in nullifying the effect of outliers.



qcut(): qcut is a quantile based discretization function that tries to divide the bins into the same frequency groups. If you try to divide a continuous variable into five bins and the number of observations in each bin will be approximately equal.



In [9]:
#name of groups
groups = ['Low', 'Med', 'High', 'Exp']

data['Item_MRP_Bin_qcut'] = pd.qcut(data['Item_MRP'], q=4, labels=groups)
data[['Item_MRP', 'Item_MRP_Bin_qcut']].head()


,Item_MRP,Item_MRP_Bin_qcut
0,249.8092,Exp
1,48.2692,Low
2,141.6180,Med
3,182.0950,High
4,53.8614,Low


In [10]:
# Count of each category 
pd.DataFrame(data['Item_MRP_Bin_qcut'].value_counts())

,Item_MRP_Bin_qcut
Exp,2131
Med,2131
Low,2131
High,2130


cut(): The cut function is also used for discretization of a continuous variable. With the qcut function, we aim to keep the number of observations in each of the bins equal, and we don’t specify where we want to make the split preferably just the number of bins required. In the case cut function, we explicitly provide the bin edges. There is no guarantee about the distribution of observation in each bin will be equal.



If we want to bin a continuous variable like age, then binning it based on frequency will not be an appropriate measure to find insights. Instead, we want to specifically divide like from 0-14 as childhood, 15-24 as a youth, and 60+ seniority. In such cases, using a cut function would make more sense than using a qcut function.



In [11]:
#define bins 
bins = [0, 70, 140, 210, 280]

#name of groups
groups = ['Low', 'Med', 'High', 'Exp']

data['Item_MRP_Bin_cut'] = pd.cut(data['Item_MRP'], bins=bins, labels=groups)

data[['Item_MRP', 'Item_MRP_Bin_cut']].head()

,Item_MRP,Item_MRP_Bin_cut
0,249.8092,Exp
1,48.2692,Low
2,141.6180,High
3,182.0950,High
4,53.8614,Low


In [12]:
# Count of each category 
pd.DataFrame(data['Item_MRP_Bin_cut'].value_counts())

,Item_MRP_Bin_cut
High,3042
Med,2778
Exp,1362
Low,1341


apply() for Text Extraction:
Pandas apply function allows to pass a function on a pandas Series and pass it to every point of the variable.

It takes a function as an argument and then applies it to either the rows or the columns of the Dataframe. We can pass any function to the argument of the apply function, but I mostly use the lambda function, which helps me write the loops and conditions in a single statement.

Using apply and lambda function, we can extract repeating credentials from the unique text present in a column. For example, we can extract titles from the given names of individuals or extract website names from the Html links. These types of signals help in improving the model performance at the model building stage.



In [13]:
data['Item_Code'] = data['Item_Identifier'].apply(lambda x: x[0:2])
data[['Item_Identifier', 'Item_Code']].head()

,Item_Identifier,Item_Code
0,FDA15,FD
1,DRC01,DR
2,FDN15,FD
3,FDX07,FD
4,NCD19,NC


value_counts() and apply() for Frequency Encoding:

If a nominal categorical variable has many categories in it, it is not recommended to use one-hot encoding. There are two primary reasons we don’t prefer one-hot encoding. First, it unnecessarily increases dimensions, and with higher dimensions, the computation time increases. The other reason is the increase in the sparsity of the one-hot encoded binary variables. The variables will have the most values as 0, which affects the performance of the model.

That’s why if we have a nominal categorical variable with a lot of categories, we prefer using frequency encoding.

Frequency Encoding is an encoding technique that encodes categorical feature values to their respected frequencies. This will preserve the information about the values of distributions. We normalize the frequencies that result in getting the sum of unique values as 1.



In [14]:
# Frequency encoding using value_counts function 
Item_Type_freq = data['Item_Type'].value_counts(normalize=True)

# Mapping the encoded values with original data 
data['Item_Type_freq'] = data['Item_Type'].apply(lambda x : Item_Type_freq[x])

print('The sum of Item_Type_freq variable:', sum(Item_Type_freq))
data[['Item_Type', 'Item_Type_freq']].head(6)

The sum of Item_Type_freq variable: 1.0


,Item_Type,Item_Type_freq
0,Dairy,0.080019
1,Soft Drinks,0.052212
2,Meat,0.049865
3,Fruits and Vegetables,0.144550
4,Household,0.106770
5,Baking Goods,0.076030


groupby() and transform() for Aggregation Features

Groupby is my go-to function to perform different tasks throughout the data analysis, transformation, and pre-processing. Groupby is a function that can split the data into various forms to get information that was not available on the surface. GroupBy allows us to group our data based on different features and get a more accurate idea about your data.

The most useful thing about the groupby function is that we can combine it with other functions like apply, agg, transform, and filter to perform tasks from data analysis to feature engineering. For our purpose, we will be using groupby with transform function to create new aggregating features.



In [15]:
data['Item_Outlet_Sales_Mean'] = data.groupby(['Item_Identifier', 'Item_Type'])['Item_Outlet_Sales']\
                                     .transform(lambda x: x.mean())

data[['Item_Identifier','Item_Type','Item_Outlet_Sales','Item_Outlet_Sales_Mean']].tail()

,Item_Identifier,Item_Type,Item_Outlet_Sales,Item_Outlet_Sales_Mean
8518,FDF22,Snack Foods,2778.3834,3232.542225
8519,FDS36,Baking Goods,549.2850,2636.568000
8520,NCJ29,Health and Hygiene,1193.1136,1221.521067
8521,FDN46,Snack Foods,1845.5976,2067.752867
8522,DRG01,Soft Drinks,765.6700,1225.072000


Series.dt() for date and time based features

Date and Time features are a gold mine for data scientists. The amount of information that we can retrieve through just one date-time variable is surprising at first, but once to get a hold of it, you’ll immediately get your hand dirty the next time we see a date-time variable in your data set.

12-07-2020 01:00:45, look at this date and think of all the possible components this particular date has. From first glance, we can tell we have a day, month, year, hour, minute, and second. But if you emphasize the date, you’ll see that you can also calculate the day of the week, a quarter of the year, week of the year, day of the year, and so on. There is no limit to the number of new variables we can create through this one date-time variable. But not every variable would be useful for the model, and using all of it means increasing dimension and even feeding noise to the model. So, it is essential to extract only those variables relevant to your data problem.

Now that we got the idea of what variables we can extract, the only thing left is to extract those features. To make this process easy, pandas have a dt function using which we can extract all the features we named above and more. I would highly recommend reading the pd.Series.dt documentation to understand what each function does.



In [ ]:
data['pickup_year'] = data['pickup_datetime'].dt.year
data['pickup_dayofyear']  = data['pickup_datetime'].dt.day
data['pickup_monthofyear'] = data['pickup_datetime'].dt.month
data['pickup_hourofday'] = data['pickup_datetime'].dt.hour
data['pickup_dayofweek'] = data['pickup_datetime'].dt.dayofweek
data['pickup_weekofyear'] = data['pickup_datetime'].dt.weekofyear